# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

import re

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
# List the databases
db_list = mongo.list_database_names()

# Check if 'uk_food' is in the list of databases
if 'uk_food' in db_list:
    print("The 'uk_food' database was created.")
else:
    print("The 'uk_food' database was not found.")
    

The 'uk_food' database was created.


In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
# List the collections in the 'uk_food' database
collection_list = db.list_collection_names()

# Display the collections in the 'uk_food' database
print("Collections in the 'uk_food' database:")
for collection in collection_list:
    print(collection)
    

Collections in the 'uk_food' database:
establishments


In [6]:
# review a document in the establishments collection
document = db.establishments.find_one()

# Display the document using pprint
pprint(document)


{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6526400a0a13a35ede9a614b'),
 'geocode': {'latitude': 51.083812, 'longitude': 1.195625},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': None

In [7]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [8]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "BusinessTypeID": "",
    "AddressLine1": "Penang Flavours",
    "AddressLine2": "146A Plumstead Rd",
    "AddressLine3": "London",
    "AddressLine4": "",
    "PostCode": "SE18 7DY",
    "Phone": "",
    "LocalAuthorityCode": "511",
    "LocalAuthorityName": "Greenwich",
    "LocalAuthorityWebSite": "http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress": "health@royalgreenwich.gov.uk",
    "scores": {
        "Hygiene": "",
        "Structural": "",
        "ConfidenceInManagement": ""
    },
    "SchemeType": "FHRS",
    "geocode": {
        "longitude": "0.08384000",
        "latitude": "51.49014200"
    },
    "RightToReply": "",
    "Distance": 4623.9723280747176,
    "NewRatingPending": True
}


In [9]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant)


In [10]:
# Check that the new restaurant was inserted
new_restaurant = establishments.find_one({"BusinessName": "Penang Flavours"})
pprint(new_restaurant)


{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RatingValue': {'$toInt': '$RatingValue'},
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6526643ddc29c42e92998e1c'),
 'geocode': {'latitude': 51.490142, 'longitude': 0.08384},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [11]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
business_type_query = {"BusinessType": "Restaurant/Cafe/Canteen"}
projection = {"BusinessTypeID": 1, "BusinessType": 1}

business_type_info = establishments.find_one(business_type_query, projection)

pprint(business_type_info)


{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('6526400a0a13a35ede9a614b')}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [12]:
# Update the new restaurant with the correct BusinessTypeID
# Define the filter to find the new restaurant
filter_query = {"BusinessName": "Penang Flavours"}

# Define the update operation to set the BusinessTypeID
update_operation = {"$set": {"BusinessTypeID": business_type_info["BusinessTypeID"]}}

# Update the new restaurant with the correct BusinessTypeID
result = establishments.update_one(filter_query, update_operation)
    

In [13]:
# Confirm that the new restaurant was updated
updated_restaurant = establishments.find_one({"BusinessName": "Penang Flavours"})

if updated_restaurant["BusinessTypeID"] == business_type_info["BusinessTypeID"]:
    print("The new restaurant was updated successfully.")
else:
    print("The new restaurant was not updated.")
    

The new restaurant was updated successfully.


In [14]:
pprint(updated_restaurant)


{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RatingValue': {'$toInt': '$RatingValue'},
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6526643ddc29c42e92998e1c'),
 'geocode': {'latitude': 51.490142, 'longitude': 0.08384},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [15]:
# Find how many documents have LocalAuthorityName as "Dover"
regex_pattern = re.compile("Dover", re.IGNORECASE)
# Count documents with LocalAuthorityName as "Dover"
dover_count = establishments.count_documents({"LocalAuthorityName": {"$regex": regex_pattern}})

# Print the number of documents with LocalAuthorityName as "Dover"
print("Number of documents with LocalAuthorityName containing 'Dover':", dover_count)


Number of documents with LocalAuthorityName containing 'Dover': 0


In [16]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = establishments.delete_many({"LocalAuthorityName": {"$regex": regex_pattern}})


In [17]:
# Check if any remaining documents include Dover
remaining_dover_count = establishments.count_documents({"LocalAuthorityName": {"$regex": regex_pattern}})


In [18]:
# Check that other documents remain with 'find_one'
# Print the number of documents deleted and remaining with LocalAuthorityName as "Dover"
print("Number of documents deleted:", delete_result.deleted_count)
print("Number of remaining documents with LocalAuthorityName containing 'Dover':", remaining_dover_count)


Number of documents deleted: 0
Number of remaining documents with LocalAuthorityName containing 'Dover': 0


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [19]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many({}, [
    {'$set': {
        'geocode.latitude': {'$toDouble': '$geocode.latitude'},
        'geocode.longitude': {'$toDouble': '$geocode.longitude'}
    }}
])


Use `update_many` to convert `RatingValue` to integer numbers.

In [20]:
# THIS CODE HAS BEEN MERGED OR REPLACED
# # Set non 1-5 Rating Values to Null
# non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
# establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [21]:
# THIS CODE HAS BEEN MERGED OR REPLACED
# # Change the data type from String to Integer for RatingValue
# establishments.update_many({"RatingValue": {"$exists": True}},
#                            [{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}])


In [22]:
# Convert RatingValue to integer numbers and set non 1-5 Rating Values to Null
# Filter out documents where RatingValue is not a valid number
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
filter_query = {
    "$or": [
        {"RatingValue": {"$in": non_ratings}},
        {"RatingValue": {"$exists": False}},
        {"RatingValue": {"$type": "number"}},
    ]
}

# Update the RatingValue field to integers for the filtered documents
update_query = {
    "$set": {"RatingValue": {"$toInt": "$RatingValue"}}
}

# Use update_many to perform the updates
result = establishments.update_many(filter_query, update_query)

# Print the result
print(f"Matched {result.matched_count} documents and modified {result.modified_count} documents.")


Matched 1 documents and modified 1 documents.


In [23]:
# Check that the coordinates and rating value are now numbers
# Find documents with numeric coordinates and RatingValue (without specifying types)
numeric_documents = establishments.find({
    "$and": [
        {"geocode.latitude": {"$exists": True}},
        {"geocode.longitude": {"$exists": True}},
        {"RatingValue": {"$exists": True}}
    ]
}).limit(5)  # Limit the results to the first 5 documents

# Iterate through the numeric documents and pprint them
for doc in numeric_documents:
    pprint(doc)
    

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6526400a0a13a35ede9a614b'),
 'geocode': {'latitude': 51.083812, 'longitude': 1.195625},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': None